In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle


In [ ]:
# this code uses the MINERvA data release and compares it to the digitized plot

# v3 gets rid of deuterium stuff for now
# does more checks on minerva

# v4 adds MINERvA uncertainties

# v5 adds deuterium with uncertainties


In [ ]:
with open('digitized_plots/minerva_digitized.csv') as f:
    lines = f.readlines()

print(lines[0])

lqcd_x = []
lqcd_y = []

lqcd_upper_x = []
lqcd_upper_y = []

lqcd_lower_x = []
lqcd_lower_y = []

deuterium_x = []
deuterium_y = []

deuterium_upper_x = []
deuterium_upper_y = []

deuterium_lower_x = []
deuterium_lower_y = []

BBB_x = []
BBB_y = []

hydrogen_x = []
hydrogen_y = []

hydrogen_upper_x = []
hydrogen_upper_y = []

hydrogen_lower_x = []
hydrogen_lower_y = []

for l in lines[2:]:
    
    nums = [s.strip() for s in l.split(",")]
    
    if nums[0] != "":
        lqcd_x.append(float(nums[0]))
        lqcd_y.append(float(nums[1]))
        
    if nums[2] != "":
        deuterium_x.append(float(nums[2]))
        deuterium_y.append(float(nums[3]))
        
    if nums[4] != "":
        BBB_x.append(float(nums[4]))
        BBB_y.append(float(nums[5]))
        
    if nums[6] != "":
        hydrogen_x.append(float(nums[6]))
        hydrogen_y.append(float(nums[7]))
    
    if nums[8] != "":
        lqcd_upper_x.append(float(nums[8]))
        lqcd_upper_y.append(float(nums[9]))
        
    if nums[10] != "":
        lqcd_lower_x.append(float(nums[10]))
        lqcd_lower_y.append(float(nums[11]))
        
    if nums[12] != "":
        deuterium_upper_x.append(float(nums[12]))
        deuterium_upper_y.append(float(nums[13]))
        
    if nums[14] != "":
        deuterium_lower_x.append(float(nums[14]))
        deuterium_lower_y.append(float(nums[15]))
        
    if nums[16] != "":
        hydrogen_upper_x.append(float(nums[16]))
        hydrogen_upper_y.append(float(nums[17]))
        
    if nums[18] != "":
        hydrogen_lower_x.append(float(nums[18]))
        hydrogen_lower_y.append(float(nums[19]))
        
    if nums[20] != "":
        hydrogen_x.append(float(nums[20]))
        hydrogen_y.append(float(nums[21]))
        
    if nums[22] != "":
        hydrogen_upper_x.append(float(nums[22]))
        hydrogen_upper_y.append(float(nums[23]))
        
sorted_zip = sorted(zip(hydrogen_x, hydrogen_y), key=lambda pair: pair[0])
hydrogen_x = [x for x, _ in sorted_zip]
hydrogen_y = [y for _, y in sorted_zip]

sorted_zip = sorted(zip(hydrogen_upper_x, hydrogen_upper_y), key=lambda pair: pair[0])
hydrogen_upper_x = [x for x, _ in sorted_zip]
hydrogen_upper_y = [y for _, y in sorted_zip]


In [ ]:
with open('digitized_plots/lqcd_digitized.csv') as f:
    lines = f.readlines()
    
lqcd_x_v2 = []
lqcd_y_v2 = []

lqcd_upper_x_v2 = []
lqcd_upper_y_v2 = []

lqcd_lower_x_v2 = []
lqcd_lower_y_v2 = []

for l in lines[2:]:
    #print(l)
    
    nums = [s.strip() for s in l.split(",")]
    
    if nums[0] != "":
        lqcd_x_v2.append(float(nums[0]))
        lqcd_y_v2.append(float(nums[1]))
    
    if nums[2] != "":
        lqcd_lower_x_v2.append(float(nums[2]))
        lqcd_lower_y_v2.append(float(nums[3]))
        
    if nums[4] != "":
        lqcd_upper_x_v2.append(float(nums[4]))
        lqcd_upper_y_v2.append(float(nums[5]))
        
print(lqcd_x_v2)
print(lqcd_y_v2)
print(lqcd_lower_x_v2)
print(lqcd_lower_y_v2)
print(lqcd_upper_x_v2)
print(lqcd_upper_y_v2)

In [ ]:
from scipy.optimize import fsolve

pion_mass = 0.135 # GeV/c^2
t_cut = 9 * pion_mass * pion_mass

def complete_a_values_8(a_values, t0, initial_guess):

    # parameters to be solved: a0, a5, a6, a7, a8,
    # equations: F(0) = -1.2723, four sum rules

    z_at_Q2_0 = (np.sqrt(t_cut + 0) - np.sqrt(t_cut - t0)) / (np.sqrt(t_cut + 0) + np.sqrt(t_cut - t0))

    a1, a2, a3, a4 = a_values

    def equations(x):
        
        F0_equation = (x[0] * z_at_Q2_0**0
                        + a1 * z_at_Q2_0**1
                        + a2 * z_at_Q2_0**2
                        + a3 * z_at_Q2_0**3
                        + a4 * z_at_Q2_0**4
                        + x[1] * z_at_Q2_0**5
                        + x[2] * z_at_Q2_0**6
                        + x[3] * z_at_Q2_0**7
                        + x[4] * z_at_Q2_0**8
                         + 1.2723)
        
        # sum from n to infinity of k(k-1)...(k-n+1) a_k = 0
        # sum from n to 8 of k(k-1)...(k-n+1) a_k = 0

        # n = 0, sum from 0 to 8 of k(k-1)...(k+1) a_k = 0
        # I guess that means sum from 0 to 8 of a_k = 0
        sum_rule_equation_0 = (x[0] + a1 + a2 + a3 + a4 + x[1] + x[2] + x[3] + x[4])

        # n = 1, sum from 1 to 8 of k(k-1)...(k) a_k = 0
        # I guess that means sum from 1 to 8 of k * a_k = 0
        sum_rule_equation_1 = (
              a1 * 1
            + a2 * 2
            + a3 * 3
            + a4 * 4
            + x[1] * 5
            + x[2] * 6
            + x[3] * 7
            + x[4] * 8
        )

        # n = 2, sum from 2 to 8 of k(k-1)...(k-1) a_k = 0
        # I guess that means sum from 2 to 8 of k(k-1) * a_k = 0
        sum_rule_equation_2 = (
              a2 * 2 * 1
            + a3 * 3 * 2
            + a4 * 4 * 3
            + x[1] * 5 * 4
            + x[2] * 6 * 5
            + x[3] * 7 * 6
            + x[4] * 8 * 7
        )

        # n = 3, sum from 3 to 8 of k(k-1)...(k-1) a_k = 0
        # I guess that means sum from 3 to 8 of k(k-1)(k-2) * a_k = 0
        sum_rule_equation_3 = (
              a3 * 3 * 2 * 1
            + a4 * 4 * 3 * 2
            + x[1] * 5 * 4 * 3
            + x[2] * 6 * 5 * 4
            + x[3] * 7 * 6 * 5
            + x[4] * 8 * 7 * 6
        )

        return np.array([F0_equation, sum_rule_equation_0, sum_rule_equation_1, sum_rule_equation_2, sum_rule_equation_3])
    
    result = list(fsolve(equations, initial_guess))

    #print("result", result)
    #print([result[0]], a_values, result[1:])
    #print("ret", [result[0]] + a_values + result[1:])
    
    return [result[0]] + list(a_values) + result[1:]



In [ ]:
# taken from https://www.nature.com/articles/s41586-022-05478-3#Sec18 supplementary information, supplementary table 4

minerva_t0 = -0.75 # GeV/c^2
minerva_a_values = [-0.50, 1.50, -1.2, -0.1, 0.2, 0.46, -0.40, 0.15, -0.044]

minerva_a_errors = [0.31, 0.7, 1.9, 3.5]
minerva_a_corr_matrix = np.array([
    [1., 0.012, -0.93, 0.52],
    [0.012, 1., -0.32, -0.78],
    [-0.93, -0.32, 1., -0.27],
    [0.52, -0.78, -0.27, 1.],
])

minerva_a_cov_matrix = np.zeros((4,4))
for i in range(4):
    for j in range(4):
        minerva_a_cov_matrix[i, j] = minerva_a_corr_matrix[i, j] * minerva_a_errors[i] * minerva_a_errors[j]

recalc_minerva_a_values = complete_a_values_8(minerva_a_values[1:5], initial_guess=minerva_a_values[0:1]+minerva_a_values[5:], t0=minerva_t0)

minerva_partial_a_universes = np.random.multivariate_normal(minerva_a_values[1:5], minerva_a_cov_matrix, 1000)
minerva_a_universes = []
for uni_i in range(len(minerva_partial_a_universes)):
    minerva_a_universes.append(complete_a_values_8(minerva_partial_a_universes[uni_i], initial_guess=minerva_a_values[0:1]+minerva_a_values[5:], t0=minerva_t0))


In [ ]:
# This is just Q^2 < 1 GeV^2, from https://journals.aps.org/prd/abstract/10.1103/PhysRevD.93.113015

deuterium_partial_a_values = [2.3, -0.6, -3.8, 2.3]
deuterium_a_errors = np.sqrt(np.array([0.0154, 1.08, 6.54, 7.40]))
deuterium_a_corr_matrix = np.array([
    [1, 0.335, -0.678, 0.611],
    [0.350, 1, -0.898, 0.367],
    [-0.678, -0.898, 1, -0.685],
    [0.611, 0.367, -0.685, 1],
])

deuterium_t0 = -0.28 # GeV/c^2

deuterium_a_cov_matrix = np.zeros((4,4))
for i in range(4):
    for j in range(4):
        deuterium_a_cov_matrix[i, j] = deuterium_a_corr_matrix[i, j] * deuterium_a_errors[i] * deuterium_a_errors[j]

deuterium_a_values = complete_a_values_8(deuterium_partial_a_values, initial_guess=[1,1,1,1,1], t0=deuterium_t0)

print("deuterium_a_values", deuterium_a_values)

deuterium_partial_a_universes = np.random.multivariate_normal(deuterium_a_values[1:5], deuterium_a_cov_matrix, 1000)
deuterium_a_universes = []
for uni_i in range(len(deuterium_partial_a_universes)):
    deuterium_a_universes.append(complete_a_values_8(deuterium_partial_a_universes[uni_i], initial_guess=[1,1,1,1,1], t0=deuterium_t0))


In [ ]:
def F_A_z2(q2, a_values, t0):

    z = (np.sqrt(t_cut + q2) - np.sqrt(t_cut - t0)) / (np.sqrt(t_cut + q2) + np.sqrt(t_cut - t0))

    ret = 0
    for i in range(len(a_values)):
        ret += a_values[i] * z**i

    return ret

def F_A_z2_func_z(z, a_values, t0):

    ret = 0
    for i in range(len(a_values)):
        ret += a_values[i] * z**i

    return ret


def F_A_dipole(q2, M_A):

    F_A_0 = -1.2723 # ± 0.0023

    return F_A_0 / (1 + q2 / (M_A**2))**2



In [ ]:
q2_vals = np.logspace(-2,np.log10(5), 100)

minerva_ratios = []
minerva_recalc_ratios = []
for q2 in q2_vals:
    minerva_ratios.append(F_A_z2(q2, minerva_a_values, minerva_t0) / F_A_dipole(q2, 1.014))
    minerva_recalc_ratios.append(F_A_z2(q2, recalc_minerva_a_values, minerva_t0) / F_A_dipole(q2, 1.014))

minerva_uni_ratio_curves = []
for a_vals in minerva_a_universes:
    minerva_uni_ratio_curves.append([])
    for q2 in q2_vals:
        minerva_uni_ratio_curves[-1].append(F_A_z2(q2, a_vals, minerva_t0) / F_A_dipole(q2, 1.014))

deuterium_ratios = []
for q2 in q2_vals:
    deuterium_ratios.append(F_A_z2(q2, deuterium_a_values, deuterium_t0) / F_A_dipole(q2, 1.014))
    
deuterium_uni_ratio_curves = []
for a_vals in deuterium_a_universes:
    deuterium_uni_ratio_curves.append([])
    for q2 in q2_vals:
        deuterium_uni_ratio_curves[-1].append(F_A_z2(q2, a_vals, deuterium_t0) / F_A_dipole(q2, 1.014))

lqcd_y_v2_ratio = []
for i in range(len(lqcd_x_v2)):
    q2 = lqcd_x_v2[i]
    fa = lqcd_y_v2[i]
    lqcd_y_v2_ratio.append(-fa / F_A_dipole(q2, 1.014))

lqcd_lower_y_v2_ratio = []
for i in range(len(lqcd_lower_x_v2)):
    q2 = lqcd_lower_x_v2[i]
    fa = lqcd_lower_y_v2[i]
    lqcd_lower_y_v2_ratio.append(-fa / F_A_dipole(q2, 1.014))

lqcd_upper_y_v2_ratio = []
for i in range(len(lqcd_upper_x_v2)):
    q2 = lqcd_upper_x_v2[i]
    fa = lqcd_upper_y_v2[i]
    lqcd_upper_y_v2_ratio.append(-fa / F_A_dipole(q2, 1.014))



In [ ]:
minerva_min_68s = []
minerva_max_68s = []
min_index = int((1 - 0.68) / 2 * len(minerva_a_universes))
max_index = int((1 + 0.68) / 2 * len(minerva_a_universes))
for ratio_vals_at_q in np.array(minerva_uni_ratio_curves).T:
    sorted_ratio_vals_at_q = sorted(ratio_vals_at_q)
    minerva_min_68s.append(sorted_ratio_vals_at_q[min_index])
    minerva_max_68s.append(sorted_ratio_vals_at_q[max_index])

deuterium_min_68s = []
deuterium_max_68s = []
min_index = int((1 - 0.68) / 2 * len(deuterium_a_universes))
max_index = int((1 + 0.68) / 2 * len(deuterium_a_universes))
for ratio_vals_at_q in np.array(deuterium_uni_ratio_curves).T:
    sorted_ratio_vals_at_q = sorted(ratio_vals_at_q)
    deuterium_min_68s.append(sorted_ratio_vals_at_q[min_index])
    deuterium_max_68s.append(sorted_ratio_vals_at_q[max_index])

In [ ]:
plt.rcParams.update({'font.size': 22})

lw = 3

plt.figure(figsize=(15,10))

plt.plot(hydrogen_x, hydrogen_y, c="tab:red", label="MINERvA (digitized from plot)", lw=lw)
plt.fill(np.append(hydrogen_upper_x, hydrogen_lower_x[::-1]), np.append(hydrogen_upper_y, hydrogen_lower_y[::-1]), c="tab:red", alpha=0.5)

plt.plot(deuterium_x, deuterium_y, c="y", label="Deuterium (digitized from plot)", lw=lw)
plt.fill(np.append(deuterium_upper_x, deuterium_lower_x[::-1]), np.append(deuterium_upper_y, deuterium_lower_y[::-1]), c="y", alpha=0.5)

plt.plot(q2_vals, minerva_ratios, label="MINERvA, kmax=8, exact values from table", lw=lw, c="tab:green")

plt.plot(q2_vals, minerva_recalc_ratios, label="MINERvA, kmax=8", lw=lw, c="tab:blue")
plt.fill_between(q2_vals, np.array(minerva_min_68s), np.array(minerva_max_68s), color="tab:blue", alpha=0.5)

plt.plot(q2_vals, deuterium_ratios, label="Deuterium, kmax=8", lw=lw, c="tab:orange")
plt.fill_between(q2_vals, np.array(deuterium_min_68s), np.array(deuterium_max_68s), color="tab:orange", alpha=0.5)

plt.xscale("log")

plt.xlabel(r"$Q^2$ (GeV/c)$^2$")
plt.ylabel(r"Ratio to dipole $F_A$ ($M_A=1.014$ GeV/c$^2$)")

plt.legend()

plt.show()

In [ ]:

#plt.plot(lqcd_x, lqcd_y, c="k", label="LQCD fit", lw=lw)
#plt.fill(np.append(lqcd_upper_x, lqcd_lower_x[::-1]), np.append(lqcd_upper_y, lqcd_lower_y[::-1]), 
#        c="k", alpha=0.2)

#plt.plot(lqcd_x_v2, lqcd_y_v2_ratio, c="k", label="LQCD fit", lw=lw)
#plt.fill(np.append(lqcd_upper_x_v2, lqcd_lower_x_v2[::-1]), np.append(lqcd_upper_y_v2_ratio, lqcd_lower_y_v2_ratio[::-1]), 
#        c="k", alpha=0.2)


In [ ]:
plt.rcParams.update({'font.size': 22})

lw = 3

plt.figure(figsize=(15,10))

plt.plot(q2_vals, minerva_recalc_ratios, label="MINERvA", lw=lw, c="tab:blue")
plt.fill_between(q2_vals, np.array(minerva_min_68s), np.array(minerva_max_68s), color="tab:blue", alpha=0.5)

plt.plot(q2_vals, deuterium_ratios, label="Deuterium", lw=lw, c="tab:orange")
plt.fill_between(q2_vals, np.array(deuterium_min_68s), np.array(deuterium_max_68s), color="tab:orange", alpha=0.5)

plt.plot(BBB_x, BBB_y, label="BBBA2007 fit", lw=lw, c="b")

plt.plot(lqcd_x_v2, lqcd_y_v2_ratio, c="k", label="LQCD fit", lw=lw)
plt.fill(np.append(lqcd_upper_x_v2, lqcd_lower_x_v2[::-1]), np.append(lqcd_upper_y_v2_ratio, lqcd_lower_y_v2_ratio[::-1]), 
        c="k", alpha=0.2)

plt.xscale("log")

plt.xlabel(r"$Q^2$ (GeV/c)$^2$")
plt.ylabel(r"Ratio to dipole $F_A$ ($M_A=1.014$ GeV/c$^2$)")

plt.legend()

plt.tight_layout()

plt.savefig("plots/F_A_plot.png", dpi=300, facecolor='w')
plt.savefig("plots/F_A_plot.jpg", dpi=300, facecolor='w')
plt.savefig("plots/F_A_plot.svg", facecolor='w')



In [ ]:
# miniboone ratio

dipole_x = np.linspace(0, 5, 1000)

miniboone_MA = 1.35
miniboone_MA_error = 0.17

miniboone_cv = []
miniboone_lower = []
miniboone_upper = []
for q2 in dipole_x:
    miniboone_cv.append(F_A_dipole(q2, miniboone_MA) / F_A_dipole(q2, 1.014))
    miniboone_lower.append(F_A_dipole(q2, (miniboone_MA - miniboone_MA_error)) / F_A_dipole(q2, 1.014))
    miniboone_upper.append(F_A_dipole(q2, (miniboone_MA + miniboone_MA_error)) / F_A_dipole(q2, 1.014))


# microboone NuWro fake data ratio

with open('trio_pickles/nuwro_fake_rate+shape_3d.pkl', 'rb') as handle:
    tup = pickle.load(handle)

    prior_trio, prior_cov = tup[0], tup[1]
    nuwro_constrained_trio, nuwro_constrained_trio_cov = tup[2], tup[3]
    nuwro_asimov_constrained_trio, nuwro_asimov_constrained_trio_cov = tup[4], tup[5]
    nuwro_cov_mat_true_trios, nuwro_cov_mat_trios, nuwro_cov_mat_trio_covs = tup[6], tup[7], tup[8]
    nuwro_xs_cov_mat_true_trios, nuwro_xs_cov_mat_trios, nuwro_xs_cov_mat_trio_covs = tup[9], tup[10], tup[11]
    nuwro_genie_true_trios, nuwro_genie_trios, nuwro_genie_trio_covs = tup[12], tup[13], tup[14]

prior_MA = prior_trio[0]
prior_MA_sigma = np.sqrt(prior_cov[0,0])
asimov_true_MA = prior_trio[0]

p = prior_MA
dp = prior_MA_sigma

nuwro_reco_MA = nuwro_constrained_trio[0]
nuwro_reco_MA_sigma = np.sqrt(nuwro_constrained_trio_cov[0,0])
x = nuwro_reco_MA
dx = nuwro_reco_MA_sigma
nuwro_reco_MA_prior_removed = (dp * dp * x - dx * dx * p) / (dp * dp - dx * dx)
nuwro_reco_MA_sigma_prior_removed = (dp * dx) / np.sqrt(dp * dp - dx * dx)

microboone_MA = nuwro_reco_MA_prior_removed
microboone_MA_error = nuwro_reco_MA_sigma_prior_removed

print(f"nuwro_reco_MA: {nuwro_reco_MA}")
print(f"nuwro_reco_MA_prior_removed: {nuwro_reco_MA_prior_removed}")

microboone_cv = []
microboone_lower = []
microboone_upper = []
for q2 in dipole_x:
    microboone_cv.append(F_A_dipole(q2, microboone_MA) / F_A_dipole(q2, 1.014))
    microboone_lower.append(F_A_dipole(q2, (microboone_MA - microboone_MA_error)) / F_A_dipole(q2, 1.014))
    microboone_upper.append(F_A_dipole(q2, (microboone_MA + microboone_MA_error)) / F_A_dipole(q2, 1.014))


In [ ]:
import uproot3

bdt_vars = [
    "nue_score",
    "numu_score",
    "numu_cc_flag"
]

eval_vars = [
    "truth_nuEnergy",
    "truth_nuPdg",
    "truth_isCC",
    "truth_vtxInside",
    "match_isFC"
]

eval_data_vars = [
    "match_isFC"
]

kine_vars = [
    "kine_reco_Enu"
]

pf_vals = [
    "reco_muonMomentum",
    "truth_muonMomentum"
]

weight_vars = [
    "weight_cv",
    "weight_spline",
    #"All_UBGenie"
]

loc = "/Users/leehagaman/data/processed_checkout_rootfiles/"

f = uproot3.open(loc + "prodgenie_bnb_nu_overlay_run1_PF/UBGenieFluxSmallUni.root")["wcpselection"]
f_bdt = f["T_BDTvars"].pandas.df(bdt_vars, flatten=False)
f_eval = f["T_eval"].pandas.df(eval_vars, flatten=False)
f_kine = f["T_KINEvars"].pandas.df(kine_vars, flatten=False)
f_pfeval = f["T_PFeval"].pandas.df(pf_vals, flatten=False)
f_weight = f["T_weight"].pandas.df(weight_vars, flatten=False)
nu_overlay_run1_pot = np.sum(f["T_pot"].pandas.df("pot_tor875good", flatten=False)["pot_tor875good"].to_numpy())
nu_overlay_run1_df = pd.concat([f_bdt, f_eval, f_kine, f_pfeval, f_weight], axis=1, sort=False)
nu_overlay_run1_df["file"] = "nu_overlay_run1"
del f
del f_bdt
del f_eval
del f_kine
del f_pfeval
del f_weight

nu_overlay_run1_df

true_muonMomentum_xs = nu_overlay_run1_df["truth_muonMomentum[0]"].to_numpy()
true_muonMomentum_ys = nu_overlay_run1_df["truth_muonMomentum[1]"].to_numpy()
true_muonMomentum_zs = nu_overlay_run1_df["truth_muonMomentum[2]"].to_numpy()
true_muonMomentum_ts = nu_overlay_run1_df["truth_muonMomentum[3]"].to_numpy()
true_Enus = nu_overlay_run1_df["truth_nuEnergy"].to_numpy() / 1000. # in GeV

true_Q2s = []
for i in range(len(true_muonMomentum_ts)):
    if true_muonMomentum_ts[i] < 0:

        true_Q2s.append(-1)

    else:

        true_Q_t = true_muonMomentum_ts[i] - true_Enus[i]
        true_Q_x = true_muonMomentum_xs[i]
        true_Q_y = true_muonMomentum_ys[i]
        true_Q_z = true_muonMomentum_zs[i] - true_Enus[i]

        true_Q2s.append(-true_Q_t**2 + true_Q_x**2 + true_Q_y**2 + true_Q_z**2)
        
nu_overlay_run1_df["true_Q2"] = np.array(true_Q2s)


In [ ]:
plt.rcParams.update({'font.size': 22})

lw = 3

plt.figure(figsize=(15,10))

plt.plot(q2_vals, minerva_recalc_ratios, label="MINERvA", lw=lw, c="tab:blue")
plt.fill_between(q2_vals, np.array(minerva_min_68s), np.array(minerva_max_68s), color="tab:blue", alpha=0.5)

plt.plot(q2_vals, deuterium_ratios, label="Deuterium", lw=lw, c="tab:orange")
plt.fill_between(q2_vals, np.array(deuterium_min_68s), np.array(deuterium_max_68s), color="tab:orange", alpha=0.5)

plt.plot(BBB_x, BBB_y, label="BBBA2007 fit", lw=lw, c="b")

plt.plot(lqcd_x_v2, lqcd_y_v2_ratio, c="k", label="LQCD fit", lw=lw)
plt.fill(np.append(lqcd_upper_x_v2, lqcd_lower_x_v2[::-1]), np.append(lqcd_upper_y_v2_ratio, lqcd_lower_y_v2_ratio[::-1]), 
        c="k", alpha=0.2)

plt.plot(dipole_x, miniboone_cv, label=r"MiniBooNE ($M_A = 1.35 \pm 0.17$)", lw=lw, c="tab:purple")
plt.fill_between(dipole_x, np.array(miniboone_lower), np.array(miniboone_upper), color="tab:purple", alpha=0.5)

plt.plot(dipole_x, microboone_cv, label=rf"MicroBooNE NuWro Fake Data ($M_A = {microboone_MA:.3f} \pm {microboone_MA_error:.3f}$)", lw=lw, c="tab:pink")
plt.fill_between(dipole_x, np.array(microboone_lower), np.array(microboone_upper), color="tab:pink", alpha=0.5)

bins = np.logspace(-2, np.log10(5), 101)
plt.hist(true_Q2s, weights=[1e-6 * len(bins) for _ in range(len(true_Q2s))], bins=bins, zorder=0, label="MicroBooNE expected $Q^2$ distribution", color="tab:gray", alpha=0.5)

plt.xscale("log")
plt.xlim(0.01, 10)
plt.ylim(0, 4)

plt.xlabel(r"$Q^2$ (GeV/c)$^2$")
plt.ylabel(r"Ratio to dipole $F_A$ ($M_A=1.014$ GeV/c$^2$)")

plt.legend()

plt.tight_layout()

plt.savefig("plots/F_A_mini_micro_plot.png", dpi=300, facecolor='w')
plt.savefig("plots/F_A_mini_micro_plot.jpg", dpi=300, facecolor='w')
plt.savefig("plots/F_A_mini_micro_plot.svg", facecolor='w')



In [ ]:
plt.rcParams.update({'font.size': 22})

lw = 3

plt.figure(figsize=(15,10))

plt.plot(q2_vals, minerva_recalc_ratios, label="MINERvA", lw=lw, c="tab:blue")
plt.fill_between(q2_vals, np.array(minerva_min_68s), np.array(minerva_max_68s), color="tab:blue", alpha=0.5)

plt.plot(q2_vals, deuterium_ratios, label="Deuterium", lw=lw, c="tab:orange")
plt.fill_between(q2_vals, np.array(deuterium_min_68s), np.array(deuterium_max_68s), color="tab:orange", alpha=0.5)

plt.plot(BBB_x, BBB_y, label="BBBA2007 fit", lw=lw, c="b")

plt.plot(lqcd_x_v2, lqcd_y_v2_ratio, c="k", label="LQCD fit", lw=lw)
plt.fill(np.append(lqcd_upper_x_v2, lqcd_lower_x_v2[::-1]), np.append(lqcd_upper_y_v2_ratio, lqcd_lower_y_v2_ratio[::-1]), 
        c="k", alpha=0.2)

plt.plot(dipole_x, miniboone_cv, label=r"MiniBooNE ($M_A = 1.35 \pm 0.17$)", lw=lw, c="tab:purple")
plt.fill_between(dipole_x, np.array(miniboone_lower), np.array(miniboone_upper), color="tab:purple", alpha=0.5)

plt.plot(dipole_x, microboone_cv, label=rf"MicroBooNE NuWro Fake Data ($M_A = {microboone_MA:.3f} \pm {microboone_MA_error:.3f}$)", lw=lw, c="tab:pink")
plt.fill_between(dipole_x, np.array(microboone_lower), np.array(microboone_upper), color="tab:pink", alpha=0.5)

bins = np.logspace(-2, np.log10(5), 101)
plt.hist(true_Q2s, weights=[1e-6 * len(bins) for _ in range(len(true_Q2s))], bins=bins, zorder=0, label="MicroBooNE expected $Q^2$ distribution", color="tab:gray", alpha=0.5)

plt.xscale("log")
plt.xlim(0.01, 3)
plt.ylim(0, 2)

plt.xlabel(r"$Q^2$ (GeV/c)$^2$")
plt.ylabel(r"Ratio to dipole $F_A$ ($M_A=1.014$ GeV/c$^2$)")

plt.legend(loc="upper left", fontsize=18)

plt.tight_layout()

plt.savefig("plots/F_A_mini_micro_plot.png", dpi=300, facecolor='w')
plt.savefig("plots/F_A_mini_micro_plot.jpg", dpi=300, facecolor='w')
plt.savefig("plots/F_A_mini_micro_plot.svg", facecolor='w')

